<a href="https://colab.research.google.com/github/Ar1stok/NLP/blob/main/sib200/sib200_ru_qwen2_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задача

- сделать анализ классификации текста с помощью LLM Qwen2-7B-Instruct
- сформировать classification_report с метриками
- замерить время получения предсказаний LLM и логрег
- сравнить метрики LLM и логрег
- описать результаты и сделать выводы

https://huggingface.co/Qwen/Qwen2-7B-Instruct - карточка модели<br>
Тут описано, что она из себя представляет и как её использовать.

In [ ]:
# установить необходимые библиотеки
!pip install fuzzywuzzy
!pip install datasets
!pip install python-Levenshtein

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.9 MB/s eta 0:00:00


In [ ]:
from typing import Dict, Union, List
from datasets import load_dataset
from fuzzywuzzy import fuzz, process
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from transformers import pipeline

In [ ]:
# функция для подбора промпта для llm
def prepare_message_for_llm(text: Union[str, List[str]], categories: List[str]) -> Dict[str, Union[List[Dict[str, str]], List[List[Dict[str, str]]]]]:
    if isinstance(text, str):
        text = [text]

    messages = []

    for msg in text:
        messages.append([{
            "role": "user",
            "content": f"Прочтите текст и определите, какая тема из списка наиболее представлена в следующем тексте. Текст: {msg} В качестве ответа напишите только название темы из списка, больше ничего: {', '.join(categories)}."
        }])

    return {'message_for_llm': messages}

In [ ]:
# Экспериментальные примеры
messages = [[{'role': 'user', 'content': 'Прочтите текст и определите, какая тема из списка наиболее представлена в следующем тексте. Текст: Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость. В качестве ответа напишите только название темы из списка, больше ничего: entertainment, geography, health, politics, science/technology, sports, travel.'}],
            [{'role': 'user', 'content': 'Прочтите текст и определите, какая тема из списка наиболее представлена в следующем тексте. Текст: Посмотрите, какие поездки рекламирует агент, на сайте или на витрине офиса. В качестве ответа напишите только название темы из списка, больше ничего: entertainment, geography, health, politics, science/technology, sports, travel.'}],
            [{'role': 'user', 'content': 'Прочтите текст и определите, какая тема из списка наиболее представлена в следующем тексте. Текст: Население Ватикана составляет около 800 человек. Это самая маленькая независимая страна в мире, а также страна, имеющая наименьшее население. В качестве ответа напишите только название темы из списка, больше ничего: entertainment, geography, health, politics, science/technology, sports, travel.'}],]

In [ ]:
# загрузка модели Qwen/Qwen2-7B-Instruct
device = 'cuda'

llm = pipeline("text-generation", model="Qwen/Qwen2-7B-Instruct", return_full_text=False,
                max_new_tokens=256, device_map='auto', torch_dtype='auto')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
# загрузка датасета по выборкам train, validation, test
dataset = load_dataset("Davlan/sib200", "rus_Cyrl")
dataset

DatasetDict({
    train: Dataset({
        features: ['index_id', 'category', 'text'],
        num_rows: 701
    })
    validation: Dataset({
        features: ['index_id', 'category', 'text'],
        num_rows: 99
    })
    test: Dataset({
        features: ['index_id', 'category', 'text'],
        num_rows: 204
    })
})

In [ ]:
# список всех категорий из всех выборок
dataset['train'] = dataset['train'].class_encode_column('category')
dataset['validation'] = dataset['validation'].class_encode_column('category')
dataset['test'] = dataset['test'].class_encode_column('category')

categories = dataset['validation'].features['category'].names
categories

['entertainment',
 'geography',
 'health',
 'politics',
 'science/technology',
 'sports',
 'travel']

In [ ]:
# добавить в список фичей колонку 'message_for_llm', которая получится в результате применения функции prepare_message_for_llm к текстам
dataset['train'] = dataset['train'].add_column(name="message_for_llm", column=prepare_message_for_llm(dataset['train']['text'], categories)['message_for_llm'])
dataset['validation'] = dataset['validation'].add_column(name="message_for_llm", column=prepare_message_for_llm(dataset['validation']['text'], categories)['message_for_llm'])
dataset['test'] = dataset['test'].add_column(name="message_for_llm", column=prepare_message_for_llm(dataset['test']['text'], categories)['message_for_llm'])

In [ ]:
dataset

In [ ]:
import time
start_time = time.time()

In [ ]:
y_pred_val = list(map(lambda x: llm(x)[0]['generated_text'], dataset['validation']['message_for_llm']))

['sports', 'travel', 'geography']

In [ ]:
# получить предсказания для валидационной выборки и сформировать classification_report
valid_pred = [categories.index(process.extractOne(pred, categories)[0]) for pred in y_pred_val]
print(classification_report(dataset['validation']['category'], valid_pred))

In [ ]:
# сделать то же самое для тестовой выборки
y_pred_test = list(map(lambda x: llm(x)[0]['generated_text'], dataset['test']['message_for_llm']))

In [ ]:
test_pred = [categories.index(process.extractOne(pred, categories)[0]) for pred in y_pred_test]
print(classification_report(dataset['test']['category'], test_pred))

In [ ]:
# замерить время получения одного предсказания и предсказаний по всему датасету, сравнить с временем получения предсказаний с помощью логрег
end_time = time.time()
print(end_time - start_time)

In [ ]:
# сделать вывод о полученных результатах
'''
    Результаты хорошие, учитывая, что не приходится тратить время на обучение, но при этом нужно уметь правильно подбирать промты.
    Также нужно учитывать, что для работы LLM нужно много видеопамяти, либо же запускать это все на сервере.
    По времени ~1027s на обработку 303 запросов, что примерно 3s на 1 запрос, в качестве чат-бота будет довольно неплох,
    так как может обрабатывать другие запросы без пере- или дообучения, т.е если у нас добавится ещё категория, результат будет похожим.
'''